# Galaxy

## Table of contents 

* [1. Getting data](#getting_data) 
* [2. Quality check](#qc1) 
* [3. Preprocessing](#preprocess) 
* [4. Quality check (2)](#qc2) 
* [5. Alignment](#alignment) 
    - [Reads alignment with Bowtie2](#align_1) 
    - [Alignment processing](#align_2) 
* [6. Variant calling](#variant_call) 
* [Galaxy Workflow and History](#galaxy_wf)

Our NGS data analysis will be performed on [Galaxy](https://usegalaxy.org), as described in the [Overview notebook](01_Overview.ipynb). You should be already logged in and ready to use this platform, so let's go ahead.

___ 

## 1. Getting data <a name="getting_data"></a>

The first thing we need for our analysis is of course retrieving some data.  
In order to keep things quick and easy, our sample will be a mitochondrial sequencing dataset: this means that only the human mitochondrial genome was sequenced in the experiment, instead of sequencing the whole genome. 

The sample will be retrieved from the [Sequence Read Archive (SRA)](https://www.ncbi.nlm.nih.gov/sra), an NCBI database which offers biological sequence data produced by researchers all around the world and made available publicly. 

In the **ToolBox** on the left, click on **Get data**, then on **EBI SRA**.  
A new view from the [ENA](https://www.ebi.ac.uk/ena) will open; click on the search bar on the top-right, and type in **SRR4420337**, then click on **Search**. 
A couple of search results will be displayed, click on the second one, which has the same ID we searched for. 

![](data/imgs/galaxy_5.jpg)

A new page with some details about this specific experiment run will show; scroll down until you reach the **Read Files** table. Since this is a paired-end sequencing experiment, we will download two `fastq` files: one containing the *forward* reads and one with the *reverse* reads.  
Click on **File 2** in the **FASTQ files (Galaxy)** column, this is the first file we need. 

![](data/imgs/galaxy_6.jpg)

Now close the Galaxy tab that just opened and go back to the ENA tab; Galaxy will keep downloading the *forward* file in the background.  
Click on **File 3** in the **FASTQ files (Galaxy)** column to download the *reverse* reads file. 

These files should now be available in your Galaxy **History**, ready to be analysed!  
If you click on the eye icon of one of these entries, you can have an overview of these files, and you should recognise that these are `fastq` files. 

___

## 2. Quality check <a name="qc1"></a>

Now we need to clean these data and make sure they are ready for the following analysis.  
For this reason, we need to perform an initial quality control using **FastQC**. 

In the **ToolBox** on the left, click on **NGS: QC and manipulation**, scroll a bit and click on **FastQC**.  
The only parameter required by FastQC is of course our sequencing data: click on the second icon, which allows to operate on multiple datasets at the same time, and select both the files we downloaded before (holding `Cmd`/`Ctrl` and clicking on each of them). 

![](data/imgs/galaxy_7.jpg)

Click on **Execute** and FastQC will be launched on our input files.  
Two outputs will be created for each input file: one with tabular, raw data, and one with a HTML representation of FastQC results. Open the latter using the eye icon the respective **History** entry. 

Look at both FastQC reports, and try to guess what preprocessing steps we might want to take before going on with our analysis. 

___

## 3. Preprocessing <a name="preprocess"></a>

Our data seems to be already in good shape, but we'll address a couple of small issues anyway; we may want to remove reads that are too short and trim their ends when the base quality is less than a specific threshold. 

In the **ToolBox**, in the **NGS: QC and manipulation** section, look for **Trimmomatic** and click on it. 

First, we should let Trimmomatic know that this is a paired-end sample, so select **Paired-end (two separate input files)** in the first dropdown menu, then select **SRR4420337_1.fastq.gz** as first input file (**R1/first of pair**) and **SRR4420337_2.fastq.gz** as second input file (**R2/second of pair**). 

![](data/imgs/galaxy_8.jpg)

In the **Perform initial ILLUMINACLIP step?** option, select **Yes**, to remove specific Illumina adapters that were used during the sequencing experiment. In the further options that will show, find **Adapter sequences to use** and choose **TruSeq3 (paired-ended, for MiSeq and HiSeq)**. Leave all the other options as they are. 

Now we can select specific preprocessing operations that will be performed. In order to address the above-mentioned issues, do the following in the **Trimmomatic Operation** section:  

- select **Cut bases off the start of a read, if below a threshold quality (LEADING)** in the dropdown menu and type **30** in the bow below it; 
- click on **Insert Trimmomatic Operation** to add another preprocessing step; 
- select **Cut bases off the end of a read, if below a threshold quality (TRAILING)** in the dropdown menu and type **30** in the box below it; 
- click on **Insert Trimmomatic Operation** to add another preprocessing step; 
- select **Drop reads below a specified length (MINLEN)** in the dropdown menu and type 40 in the bow below it. 

![](data/imgs/galaxy_9.jpg)

Now click on **Execute** and let it work.  
Trimmomatic will produce 4 output files: the first two contain all reads from R1 and R2 that, after the preprocessing step, maintain their mate read (**R1 paired** and **R2 paired**, respectively), while the other two contain reads that lost their mate during the preprocessing (**R1 unpaired** and **R2 unpaired**), and we will ignore them. 

___

## 4. Quality check (2) <a name="qc2"></a>

After cleaning our data, we should be ready to start our actual analysis. Let's perform another quality check with FastQC to be sure that everything went smoothly.  
Repeat what was done in the preliminary quality check (see **2. Quality check**), but this time be sure to use the output files produced by Trimmomatic, specifically **R1 paired** and **R2 paired**. 

You should notice that our data looks much cleaner now! 

___

## 5. Alignment <a name="alignment"></a>

We can finally align our reads against a reference genome.  
We will use [Bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) to perform the alignment, and the hg38 genome assembly as reference genome. The mitochondrial reference genome contained in this genome assembly is the [revised Cambridge Reference Sequence (rCRS)](https://mitomap.org//bin/view.pl/MITOMAP/HumanMitoSeq). 

### Reads alignment with Bowtie2 <a name="align_1"></a>

In the **ToolBox**, look for the **NGS: Mapping** section, then select **Bowtie2**.  
Select **Paired-end** in the library type option, and select the forward and reverse Fastq files (respectively the **R1 paired** and **R2 paired** outputs of Trimmomatic).  
In the **Select a reference genome** option, select **hg38 Canonical** (hg38 contains all canonical chromosomes as well as unplaced contigs, while hg38 Canonical only contains chromosomes from 1 through 22, X, Y and MT).  
Leave all the other options untouched and click on **Execute**. 

The output of this step is a [BAM](https://support.illumina.com/help/BS_App_RNASeq_Alignment_OLH_1000000006112/Content/Source/Informatics/BAM-Format.htm) file, which is binary and cannot be viewed unless transformed to a more human-readable [SAM](https://en.wikipedia.org/wiki/SAM_(file_format)) format. But, thanks to the magic of Galaxy, we can still have a look at this file without performing any format conversion: just click on the eye icon to have an overview of the alignment results. 

You should see that the first rows of this file are reads that were aligned on chromosome 1. But we used a sample which was specifically from a mitochondrial sequencing experiment, so how is this possible?  
Most of the mitochondrial reads mapping on different chromosomes are [NumtS](https://en.wikipedia.org/wiki/NUMT), which are pieces of mitochondrial genome that were relocated in a different genomic position during the evolution; an extensive list of these NumtS can be found [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3228558/). 

### Alignment processing <a name="align_2"></a>

So we need to get rid of all these NumtS and keep only reads mapped on the mitochondrial chromosome. In order to do that, in the **ToolBox** select the **NGS: SAMtools** section, then click on **Filter SAM or BAM, output SAM or BAM**.  
Select the output of the previous step as input, then in the **Select regions** option type in **chrM**, and click on **Execute**. 

![](data/imgs/galaxy_10.jpg)

If you have a look at the output file just produced, you should see that only reads mapping on chrM were retained. 

___

## 6. Variant calling <a name="variant_call"></a>

We can now exploit the aligned reads to identify variations with respect to the human mitochondrial reference genome; this is the variant calling step.  
In **NGS: Variant Analysis** in the **ToolBox**, click on **Naive Variant Caller (NVC)**; select the filtered BAM file as input, then select **hg38** as reference genome.  
We can apply some further options to optimize the variant calling:  

- in **Minimum number of reads needed to consider a REF/ALT**, insert **50**; 
- in **Minimum base quality**, insert **28**; 
- in **Minimum mapping quality**, insert **18**; 
- in **Ploidy**, insert **1**; 
- select **Yes** in **Only write out positions with possible alternate alleles**; 
- select **No** in **Report counts by strand**. 

![](data/imgs/galaxy_11.jpg)

Click on **Execute** to launch the variant calling process. 

The output of this step is a VCF file, which should contain ~90 variants found in our sample. As usual, you can have a look at this file by clicking on the eye icon in the related **History** entry. 

We found some variants in our sample, but we may want to know whether these are only polymorphisms or they are actually involved in any disease. The **functional annotation** of these variants will be performed outside Galaxy, since it does not offer efficient tools for mitochondrial variants annotation.  
In order to do this, we must download the VCF file just obtained. Click on the title of the last analysis step performed to show its details, and click on the floppy disk icon to save the file. 

![](data/imgs/galaxy_15.jpg)

Save this file somewhere with a filename you can remember. We will need this VCF file in the following steps with RStudio.  
Now go to the [RStudio notebook](03_RStudio.ipynb) to continue our analysis. 

___

## Galaxy Workflow and History <a name="galaxy_wf"></a>

Analyses conducted on Galaxy can be shared and rerun using a Workflow or a History. There is a subtle difference between these two options:  

- workflows only save processing steps, with no information about the actual samples used in the analysis. This allows to reproduce the very same analysis also on different samples. 
- histories save an entire analysis comprehensive of input and output files created. You won't be able to use different input files, unless you modify the history or rerun one or more jobs on specific samples. 

The entire analysis conducted above is available in this [Galaxy Workflow](https://usegalaxy.org/u/robertopreste/w/ngsdatapracticeworkflow).  
If you click on the green plus icon in the upper right corner, you can import this workflow in your Galaxy session. Before using it, you will have to download some sample data from SRA, or you can use the SRR4420337 sample downloaded before. This is because Galaxy does not allow to download automatically SRA samples in a workflow. 

You can also access this analysis from the history, which stores all the analysis steps as well as objects created. 
The entire analysis conducted above is available in this [Galaxy History](https://usegalaxy.org/u/robertopreste/h/ngsdatapractice). You can click on **Import history** in the top right to import this history into your Galaxy session. 